### Obliczanie uzysku energii z instalacji fotowoltaicznej

Obliczenia dla instalacji realizowane są na podstawie najnowszych wyników badań natężenia promieniowania słonecznego realizowanego przez EUMETSAT Climate Monitoring Satellite Application Facility (CM SAF). Dane te tworzą bazę PVGIS-SARAH2 udostępnianą przez EU Science HUB. Informacje o bazie i zapisywanych w niej danych można znaleźć na stronie: https://joint-research-centre.ec.europa.eu/pvgis-photovoltaic-geographical-information-system/pvgis-data-download/sarah-solar-radiation-data_en

Po uruchomieniu modułu uruchomiony zostaje panel konfiguracyjny instalacji dla czterech jej części. Pozostawienia ilości paneli z wartością zero wyłącza z obliczeń daną część. Pozostałe parametry są zgodne z opisem przedstawionym we wpisie.

Pozdrawiam i życzę udanego korzystania

Jurand Bień

In [1]:
import pandas as pd

import matplotlib.pyplot as plt
import pvlib

import ipywidgets as widgets
from ipywidgets import HBox, VBox, Button
from IPython.display import clear_output

In [2]:
def obtain_pv_power_from_PVGIS(lat, lon, start, end, trackingtype, name, 
                               tilt, azimuth, panels, peakpower, pvtech, mplace):
    
    if panels > 0:
        poa, _, _ = pvlib.iotools.get_pvgis_hourly(latitude = lat,
                                                   longitude = lon,
                                                   start = start,
                                                   end = end,
                                                   raddatabase = 'PVGIS-SARAH2',
                                                   components = True,
                                                   surface_tilt = tilt,
                                                   surface_azimuth = azimuth,
                                                   pvcalculation = True,
                                                   peakpower = panels*peakpower,
                                                   pvtechchoice = pvtech,
                                                   mountingplace = mplace,
                                                   loss = 10,
                                                   trackingtype = trackingtype,
                                                   url='https://re.jrc.ec.europa.eu/api/v5_2/', 
                                                  )
   
    else:
        poa, _, _ = pvlib.iotools.get_pvgis_hourly(latitude = lat,
                                                   longitude = lon,
                                                   start = start,
                                                   end = end,
                                                   raddatabase = 'PVGIS-SARAH2',
                                                   components = True,
                                                   surface_tilt = tilt,
                                                   surface_azimuth = azimuth,
                                                   pvcalculation = False,
                                                   url='https://re.jrc.ec.europa.eu/api/v5_2/', 
                                                  )
        poa['P'] = 0.0

    poa['date'] = pd.to_datetime(poa.index.date)
    poa['P'] = poa['P'].div(1000)
    poa = poa[['date','P']].groupby('date').sum()
    poa['location'] = name 
    return poa

def get_data_for_pv_installation(lat, lon, pv_conf):
    
    starting_time = pd.Timestamp('2020-01-01')
    ending_time = pd.Timestamp('2020-12-31')
    trackingtype = 0
    
    poas = {}
    poalist = []
    
    for pv_part in pv_conf:
        poa = obtain_pv_power_from_PVGIS(lat, lon, starting_time, ending_time, trackingtype,
                                         pv_part['name'], pv_part['tilt'], pv_part['azimuth'], 
                                         pv_part['num_panels'], pv_part['peak_power'], 
                                         pv_part['tech_pv'], pv_part['mounting'])
        
        poas[pv_part['name']] = poa
        poalist.append(poa)
    
    dataset = pd.concat(poalist).sort_index()
    dataset['month'] = dataset.index.month
    return dataset

def create_pv_configuration(nazwa,nach_inst,azymut,ilosc_paneli,moc_poj_panelu,tech_panel,mon_panel):
    pv_conf = []
    for n,t,a,np,pp,tp,mp in zip(nazwa,nach_inst,azymut,ilosc_paneli,moc_poj_panelu, tech_panel, mon_panel):
        pv_conf.append({'name':n, 'tilt':t, 'azimuth':a, 'num_panels':np,
                             'peak_power':pp, 'tech_pv':tech_panel, 'mounting': mon_panel})
    return pv_conf

def create_expanded_button(description, button_style):
    return Button(description=description, button_style=button_style, layout=widgets.Layout(height='auto', width='auto'))

def addlabels(x,y,ax):
    for i in range(len(x)):
        ax.text(i+1,y[i]//2,round(y[i],1),ha='center',fontsize=6,bbox=dict(facecolor = 'white', alpha = .6))

def plot_months(pv_calc_df):
    
    df1 = pv_calc_df[['month','P']].groupby('month').sum().reset_index()
    
    fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(10,10))
    axes[0].bar(df1.month,df1.P)
    addlabels(df1.month,df1.P,axes[0])
    axes[1].plot(df1.month, df1.P.cumsum())
    
    value = df1.P.mean()
    axes[0].axhline(value, linestyle='--', color='green')
    axes[0].annotate('%0.0f' % value, xy=(1, value), xytext=(-40, 10), xycoords=('axes fraction', 'data'), textcoords='offset points')
    
    value = df1.P.cumsum().max()
    axes[1].annotate('%0.0f' % value, xy=(1, value), xytext=(-40, -20), xycoords=('axes fraction', 'data'), textcoords='offset points')
    
    axes[0].set_title('Obliczona, wytworzona ilość energii [kWh]')
    axes[1].set_title('Obliczona, skumulowana ilość energii w okresie roku [kWh]')
    axes[0].set_xlabel('Miesiąc')
    axes[1].set_xlabel('Miesiąc')
    axes[0].set_ylabel('kWh')  
    
    plt.show()
    return None

In [3]:
#button_calculate = create_expanded_button('Oblicz','success')

button_calculate = widgets.Button(
    description='Uruchom obliczenia',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Obliczanie uzysku energii elektrycznej z instalacji',
    icon='check', # (FontAwesome names without the `fa-` prefix)
    layout=widgets.Layout(height='auto', width='auto')
)

def on_button_clicked(event):
    
    nazwa = [names['nazwa_inst%01d' % x].value for x in range(1,5)]
    nach_inst = [angles['nach_inst%01d' % x].value for x in range(1,5)]
    azymut = [azimuths['azymut_inst%01d' % x].value for x in range(1,5)]
    ilosc_paneli = [pv_counts['il_inst%01d' % x].value for x in range(1,5)]
    moc_poj_panelu = [pv_power['moc_panel%01d' % x].value for x in range(1,5)]
    tech_panel = [pv_technology['tech_panel%01d' % x].value for x in range(1,5)]
    mon_panel = [pv_mounting['mon_panel%01d' % x].value for x in range(1,5)]

    pv_configuration = create_pv_configuration(nazwa,nach_inst,azymut,ilosc_paneli,moc_poj_panelu,tech_panel,mon_panel)
    
    with output:
        clear_output()
        print("Wykonuję obliczenia, proszę o chwilę cierpliwości")
        pv_df = get_data_for_pv_installation(szer, dlug, pv_configuration)
        clear_output()
        plot_months(pv_df)

output = widgets.Output()
button_calculate.on_click(on_button_clicked)

In [4]:
# konfiguracja widgetów i wygladu strony

# nagłowek
conf_headline = create_expanded_button('KONFIGURACJA INSTALACJI FOTOWOLTAICZNEJ','success')

#panel lewy
file = open('logo.png','rb')
image = file.read()
image_png = widgets.Image(value = image,
                               format='png',
                               width='140')
vbox_image = widgets.VBox([image_png]) 

#wspolrzedne geograficzne
vbox_local = widgets.HTML(value = "<br/>Lokalizacja instalacji<hr>Szerokość geograficzna")
vbox_szer_geo = widgets.BoundedFloatText(value=50.743124,
                                    min=48,
                                    max=55,step=0.000001,
                                    description='',
                                    disabled=False,
                                    style={'description_width': 'initial'},
                                    layout = widgets.Layout(width='140px'))
szer = vbox_szer_geo.value
vbox_dlug_label = widgets.HTML(value = "Długość geograficzna")
vbox_dlug_geo = widgets.BoundedFloatText(value=17.908828,
                                    min=13.9,
                                    max=24.2,step=0.000001,
                                    description='',
                                    disabled=False,
                                    style={'description_width': 'initial'},
                                    layout = widgets.Layout(width='140px'))
dlug = vbox_dlug_geo.value
vbox_break = widgets.HTML(value = "&emsp;&emsp;&emsp;")
vbox_localization = widgets.VBox([vbox_image,vbox_local,vbox_szer_geo,vbox_dlug_label,vbox_dlug_geo])

#konfiguracja instalacji

nazwa_inst0 = widgets.Label(value='Nazwa:')
nach_inst0 = widgets.Label(value='Kąt nachylenia:')
azymut_inst0 = widgets.Label(value='Azymut:')
il_inst0 = widgets.Label(value='Ilość paneli:')
moc_panel0 = widgets.Label(value='Moc panelu:')
tech_panel0 = widgets.Label(value='Technologia:')
mon_panel0 = widgets.Label(value='Montaż:')
panel0 = widgets.VBox([nazwa_inst0, nach_inst0, azymut_inst0, il_inst0, moc_panel0, tech_panel0, mon_panel0])

names = dict()
angles = dict()
azimuths = dict()
pv_counts = dict()
pv_power = dict()
pv_technology = dict()
pv_mounting = dict()

for i in range(1,5):
    names['nazwa_inst%01d' % i ] = widgets.Text(value='part_of_pv',placeholder='Part of pv installation',
                                                description='',disabled=False,
                                                style={'description_width': 'initial'},
                                                layout = widgets.Layout(width='140px'))
    angles['nach_inst%01d' % i ] = widgets.BoundedIntText(value=35,min=0,max=90,step=1,
                                                          description='',disabled=False,
                                                          style={'description_width': 'initial'},
                                                          layout = widgets.Layout(width='140px'))
    azimuths['azymut_inst%01d' % i ] = widgets.BoundedIntText(value=0,min=-180,max=180,step=1,
                                                              description='',disabled=False,
                                                              style={'description_width': 'initial'},
                                                              layout = widgets.Layout(width='140px'))
    pv_counts['il_inst%01d' % i ] = widgets.BoundedIntText(value=0,min=0,max=50,step=1,
                                                           description='',disabled=False,
                                                           style={'description_width': 'initial'},
                                                           layout = widgets.Layout(width='140px'))
    pv_power['moc_panel%01d' % i ] = widgets.BoundedFloatText(value=0.330,min=0,max=1,step=0.01,
                                                              description='',disabled=False,
                                                              style={'description_width': 'initial'},
                                                              layout = widgets.Layout(width='140px'))
    pv_technology['tech_panel%01d' % i] = widgets.Dropdown(options=['crystSi', 'CIS', 'CdTe', 'Unknown'],value='crystSi',
                                                           description='',disabled=False,
                                                           style={'description_width': 'initial'},
                                                           layout = widgets.Layout(width='140px'))
    pv_mounting['mon_panel%01d' % i] = widgets.Dropdown(options=['free', 'building'],value='building',
                                                        description='',disabled=False,
                                                        style={'description_width': 'initial'},
                                                        layout = widgets.Layout(width='140px'))

panel1 = widgets.VBox([names['nazwa_inst1'], angles['nach_inst1'], azimuths['azymut_inst1'], 
                       pv_counts['il_inst1'], pv_power['moc_panel1'], pv_technology['tech_panel1'], 
                       pv_mounting['mon_panel1']])  
panel2 = widgets.VBox([names['nazwa_inst2'], angles['nach_inst2'], azimuths['azymut_inst2'], 
                       pv_counts['il_inst2'], pv_power['moc_panel2'], pv_technology['tech_panel2'], 
                       pv_mounting['mon_panel2']]) 
panel3 = widgets.VBox([names['nazwa_inst3'], angles['nach_inst3'], azimuths['azymut_inst3'], 
                       pv_counts['il_inst3'], pv_power['moc_panel3'], pv_technology['tech_panel3'], 
                       pv_mounting['mon_panel3']]) 
panel4 = widgets.VBox([names['nazwa_inst4'], angles['nach_inst4'], azimuths['azymut_inst4'], 
                       pv_counts['il_inst4'], pv_power['moc_panel4'], pv_technology['tech_panel4'], 
                       pv_mounting['mon_panel4']]) 

pv_conf = widgets.HBox([panel0,panel1,panel2,panel3,panel4])
pv_calc = widgets.VBox([pv_conf,button_calculate])

panel_konf = widgets.HBox([vbox_localization,vbox_break,pv_calc])
vbox_pv_configuration = widgets.VBox([conf_headline,panel_konf])
upper = widgets.HBox([vbox_pv_configuration])

In [5]:
page = widgets.VBox([upper, output])
display(page)